In [ ]:
# dataset link : https://huggingface.co/datasets/thananos/augmented-scam-dialogue

In [ ]:
import pandas as pd

test_df = pd.read_parquet('test-00000-of-00001.parquet')
train_df = pd.read_parquet('train-00000-of-00001.parquet')

In [ ]:
import torch
# GPU Code
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using :',device)

Using : cuda


In [ ]:
x_train = train_df['dialogue']
y_train = train_df['label']
x_test = test_df['dialogue']
y_test = test_df['label']

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

In [ ]:
train_corpus = " [NEW] ".join(x_train)
print(len(train_corpus.split()))
print(len(set(train_corpus.split())))

1277252
11687


In [ ]:
test_cropus = " [NEW] ".join(x_test)
print(len(test_cropus.split()))
print(len(set(test_cropus.split())))

78184
4128


In [ ]:
corpus = train_corpus + " [NEW] " + test_cropus
print(len(corpus.split()))
print(len(set(corpus.split())))

1355437
12207


In [ ]:
import string,re

def preprocess(text):
  text = text.strip().lower()

  text = text.replace('%','percentage ')
  text = text.replace('$','dollar ')
  text = text.replace('₹','rupee ')
  text = text.replace('€','euro ')
  text = text.replace('@','at ')
  text = text.translate(str.maketrans('', '', '!#$%&()*+,-./:;<=>?@[\]^_{|}~'))
  text = text.replace("’", "'").replace("‘", "'")

  contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

  for i in text.split():
    if i in contractions:
      text = text.replace(i,contractions[i])

  return " ".join(text.split())

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

def lemmatize_text(text):
    doc = nlp(text)
    out_text = []
    for token in doc:
        if not token.is_punct and not token.is_space:
            out_text.append(token.lemma_)
    return " ".join(out_text)


In [ ]:
x_train =x_train.apply(preprocess)
x_test = x_test.apply(preprocess)

In [ ]:
x_train = x_train.apply(lemmatize_text)
x_test = x_test.apply(lemmatize_text)

In [ ]:
train_corpus = " [NEW] ".join(x_train)
test_cropus = " [NEW] ".join(x_test)
corpus = train_corpus + " [NEW] " + test_cropus

In [ ]:
from nltk.tokenize import word_tokenize

def tokenize(word):
  return word_tokenize(word)

In [ ]:
vocab = {
    '<UNK>' : 0,
}

for i in corpus.split('[NEW]'):
  for j in i.split(" "):
    if j not in vocab:
      vocab[f'{j}'] = len(vocab)

In [ ]:
vocab

{'<UNK>': 0,
 'caller': 1,
 'hello': 2,
 'be': 3,
 'this': 4,
 'john': 5,
 'receiver': 6,
 'yes': 7,
 'it': 8,
 'who': 9,
 'call': 10,
 'my': 11,
 'name': 12,
 'officer': 13,
 'johnson': 14,
 'from': 15,
 'the': 16,
 'social': 17,
 'security': 18,
 'administration': 19,
 'how': 20,
 'you': 21,
 'today': 22,
 'I': 23,
 'fine': 24,
 'thank': 25,
 'what': 26,
 'can': 27,
 'do': 28,
 'for': 29,
 'we': 30,
 'have': 31,
 'try': 32,
 'to': 33,
 'reach': 34,
 'about': 35,
 'a': 36,
 'very': 37,
 'important': 38,
 'matter': 39,
 'your': 40,
 'number': 41,
 'compromise': 42,
 'and': 43,
 'need': 44,
 'take': 45,
 'immediate': 46,
 'action': 47,
 'protect': 48,
 'identity': 49,
 'oh': 50,
 'no': 51,
 'happen': 52,
 'receive': 53,
 'report': 54,
 'of': 55,
 'suspicious': 56,
 'activity': 57,
 'on': 58,
 'account': 59,
 'verify': 60,
 'some': 61,
 'information': 62,
 'ensure': 63,
 'benefit': 64,
 'not': 65,
 'interrupt': 66,
 'okay': 67,
 'know': 68,
 'please': 69,
 'confirm': 70,
 'wait': 71,
 'c

In [ ]:
len(vocab)

6768

In [ ]:
def words_num(text):
  conv = []
  for i in text.split(" "):
    if i in vocab:
      conv.append(vocab[i])
    else:
      conv.append(vocab['<UNK>'])
  return conv

In [ ]:
x_train_num = x_train
x_test_num = x_test

In [ ]:
x_train_num = x_train_num.apply(words_num)
x_test_num = x_test_num.apply(words_num)

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec

In [ ]:
sentence = []
for i in x_train:
  sentence.append(list(i.split(" ")))
model = Word2Vec(sentences=sentence,vector_size=200,window=5,min_count=1,sg=1)

model.save("fraud_word2vec.model")

In [ ]:
test_sentence = []

for i in x_test:
  test_sentence.append(list(i.split(" ")))

In [ ]:
import numpy as np
embedding_dim = 200
embedding_matrix = np.zeros((len(vocab),embedding_dim))

for word,idx in vocab.items():
  if word in model.wv:
    embedding_matrix[idx] = model.wv[word]
  else:
    embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

In [ ]:
len(max(sentence)),len(max(test_sentence))

(143, 241)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train_num_pad = pad_sequences(x_train_num, maxlen=250, padding='post')
x_test_num_pad =  pad_sequences(x_test_num, maxlen=250, padding='post')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=len(vocab),
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=250,
                    trainable=False))

model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, 250))
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 250, 200)       │     1,353,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 128)            │       168,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,522,177 (5.81 MB)

 Trainable params: 168,577 (658.50 KB)

 Non-trainable params: 1,353,600 (5.16 MB)

In [ ]:
model.fit(x_train_num_pad, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.6341 - loss: 0.5961 - val_accuracy: 0.1937 - val_loss: 0.7003
Epoch 2/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6754 - loss: 0.5183 - val_accuracy: 0.5602 - val_loss: 0.6384
Epoch 3/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7341 - loss: 0.5280 - val_accuracy: 0.8562 - val_loss: 0.3536
Epoch 4/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8592 - loss: 0.3152 - val_accuracy: 0.9258 - val_loss: 0.1546
Epoch 5/5
160/160 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9511 - loss: 0.1620 - val_accuracy: 0.9312 - val_loss: 0.1969


In [ ]:
model.export('fraud_NN.keras')

Saved artifact at 'fraud_NN.keras'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 250), dtype=tf.float32, name='keras_tensor_40')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136978010966160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136977985992272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136977985990160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136977985989968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136977985992656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136977985989392: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
model.save("fraud_lstm_model.h5")

In [ ]:
model.evaluate(x_test_num_pad, y_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9930 - loss: 0.0740


[0.0817403793334961, 0.987500011920929]

In [1]:

fraud_call_text = """
Caller: Hi, this is Chris from the Billing Department at American Power & Electric. I’m calling because your last payment didn’t process correctly, and your electric service is scheduled for disconnection in the next two hours.

Receiver: What? I paid my bill last week.

Caller: I understand, and that’s why I’m calling you before the cutoff happens. Sometimes the online system doesn’t update right away. To avoid losing power, we can process your payment over the phone right now. Once it’s verified, the disconnection will be canceled immediately.

Receiver: I usually pay online. Can’t I just do that?

Caller: Unfortunately, the automated system has already locked the account, so online payments won’t post in time. The only way to stop the service interruption is through a same-day manual payment. We’re accepting a secure electronic voucher or prepaid card that you can buy at any local store, like Walmart or CVS. Once you have the card, you give us the number, and your account will be cleared.

Receiver: That sounds strange. Why can’t I just call the company directly?

Caller: If you call the main line, the disconnection will continue because it takes 24–48 hours to process. I’m giving you a direct option to stop it immediately. It’s your choice, but if the technician arrives, there’s a $75 reconnection fee on top of your balance. Do you want to proceed and keep your power on?
"""

In [ ]:
input_str = preprocess(fraud_call_text)
input_str = lemmatize_text(input_str)
input_str = words_num(input_str)
input_str = pad_sequences([input_str], maxlen=250, padding='post')

prediction  = model.predict(input_str)
print(prediction)
if(prediction>0.5):
  print("Fraudulent")
else:
  print("Not Fraudulent")

In [ ]:
import pickle

with open("vocab_dict.pkl", "wb") as f:
    pickle.dump(vocab, f)
